## Trabajo practico N° 1: Algoritmos de busqueda en Torre de Hanoi

### Integrantes: Maxim Dorogov

1. **¿Cuáles son los PEAS de este problema? (Performance, Environment, Actuators, Sensors)**

- **Performance**: 

    Para el problema de las torres de hanoi podemos definir el rendimiento como número de movimientos realizados para resolver el problema. Un menor número de movimientos indica una solución con mejor rendimiento.
- **Environment**: 

    El entorno es el sistema compuesto por las torres y los discos.
- **Actuators**: 

    Los actuadores son los discos que se mueven entre las torres.
- **Sensors**: 

    En este problema al ser un entorno observable se conoce en todo momento el estado real. Estimo que los sensores serian los ojos del agente, que le permiten observar el estado de las torres y los discos.

2. **¿Cuáles son las propiedades del entorno de trabajo?**
Es un entorno observable, dinamico y secuencial. Ya que los discos cambian de posicion con cada movimiento pero lo hacen siguiendo un orden establecido definiendo secuencias de estados.

3. **En el contexto de este problema, defina los siguientes conceptos:**

- **Estado**: 

Configuracion de los discos en las torres en un instante determinado.
- **Espacio de estados**: 

Todas las combinaciones posibles de los discos en las torres.
- **Árbol de búsqueda**: 

Representación gráfica de todas las combinaciones posibles de los discos en las torres y los movimientos realizados para lograr cada combinacion.
- **Nodo de búsqueda**: 

El nodo de búsqueda es una representación de un estado en el árbol de búsqueda. Contiene información sobre la configuración de los discos en las torres y el camino recorrido para llegar a ese estado.
- **Objetivo**: 

Un estado en el que todos los discos se encuentran en la torre de destino o la configuracion que uno proponga como solucion final.
- **Acción**: 

Mover un disco de una torre a otra.
- **Frontera**: 

La frontera es la separacion de dos regiones del grafo, los estados que fueron explorados y los que no.

4. **Implemente algún método de búsqueda. Podés elegir cualquiera excepto búsqueda en anchura (breadth-first search), que ya fue desarrollada en clase. Sos libre de utilizar cualquiera de los algoritmos vistos, o incluso explorar nuevos.**

Se decidio llevar a cabo la resolucion del problema de las torres de Hanoi mediante busqueda informada (Greedy).
Se utilizo el costo de llegar a un nodo como la funcion heuristica.

Una vez implementado el algoritmo greedy se realizo una comparacion con la implementacion de busqueda primero en anchura que se mostro en clase.

5. **¿Cuál es la complejidad teórica en tiempo y memoria del algoritmo elegido?**

Para greedy la complejidad en tiempo es O(b^d) donde b es el el numero promedio de ramificaciones por cada nodo y d es la profundidad del arbol.


**Items 6 y 7 se responden en la seccion de metricas del notebook**

## Solucion propuesta

In [2]:
from queue import PriorityQueue
from typing import Tuple, Union, Set
import tracemalloc

from aima_libs.tree_hanoi import NodeHanoi
from aima_libs.hanoi_states import (
    ProblemHanoi, 
    StatesHanoi,
)

### Algoritmo de busqueda informada Greedy

In [7]:
def greedy_hanoi_solver(
    number_disks: int = 5
) -> Union[Tuple[int, Set[StatesHanoi], StatesHanoi, PriorityQueue], None]:
    """
    Informed search greedy solver for the Tower of Hanoi problem.

    Parameters
    ----------
    number_disks : int
        Number of disks in the Tower of Hanoi problem.
    
    Returns
    -------
    Tuple[int, Set[StatesHanoi], StatesHanoi, PriorityQueue] or None
        Number of nodes explored, the traveled nodes and the goal state if 
        solution is found otherwise None.
    """
    disks = [i for i in range(number_disks, 0, -1)] # N, N-1, N-2, ..., 1

    # All disks are in the first rod at the beginning
    initial_state = StatesHanoi(
        rod1=disks, rod2=[], rod3=[], max_disks=number_disks)

    # All disks must be in the third rod at the end
    goal_state = StatesHanoi(
        rod1=[], rod2=[], rod3=disks, max_disks=number_disks)
    
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)
    initial_node = NodeHanoi(state=initial_state)

    frontier = PriorityQueue() 
    frontier.put((initial_node.path_cost, initial_node))
    
    # Will store the states that we have already explored
    explored_states = set() 

    explored_ctr = 0
    solution = None

    while not frontier.empty():

        # Get a node from the frontier and increment the ctr of explored nodes
        _, node = frontier.get()
        explored_ctr += 1
        explored_states.add(node.state)        

        if problem.goal_test(node.state):
            solution = (explored_ctr, explored_states, node, frontier)
            # solution found exit the loop
            break
        
        # Add to queue all the not expored successors of the current node
        for next_node in node.expand(problem):
            if next_node.state not in explored_states:
                frontier.put((next_node.path_cost, next_node))
    return solution

### Solucion implementada en clase de busqueda en anchura

In [6]:
def breadth_first_search(number_disks=5):
    # Inicializamos el problema
    list_disks = [i for i in range(number_disks, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    frontier = [NodeHanoi(problem.initial)] # Cola FIFO con el nodo inicial
    explored = set() # Conjunto de estados ya visitados

    node_explored = 0
    
    while len(frontier) != 0:
        node = frontier.pop()
        node_explored += 1
        
        explored.add(node.state) # Verificamos si llegamos al objetivo
        
        if problem.goal_test(node.state):
            metrics = {
                "solution_found": True,
                "nodes_explored": node_explored,
                "states_visited": len(explored),
                "nodes_in_frontier": len(frontier),
                "max_depth": node.depth,
                "cost_total": node.state.accumulated_cost,
            }
            return node, metrics
        
        # Agregamos a la frontera los nodos sucesores que no hayan sido visitados
        for next_node in node.expand(problem):
            if next_node.state not in explored:
                frontier.insert(0, next_node)

    # Si no se encuentra solución, devolvemos métricas igualmente
    metrics = {
        "solution_found": False,
        "nodes_explored": node_explored,
        "states_visited": len(explored),
        "nodes_in_frontier": len(frontier),
        "max_depth": node.depth, # OBS: Si no se encontró la solución, este valor solo tiene sentido en breadth_first_search, en otros casos se debe ir llevando registro de cual fue la máxima profundidad
        "cost_total": None,
    }
    return None, metrics

6. **A nivel de implementación, ¿cuánto tiempo y memoria utiliza el algoritmo? (Se recomienda ejecutarlo 10 veces y calcular el promedio y el desvío estándar de ambas métricas).**
### Metricas y utilizacion de recursos para ambas soluciones

In [8]:
DISKS_NUMBER = 5

# -- Greedy solver
solution = greedy_hanoi_solver(number_disks=DISKS_NUMBER)

if solution is not None:
    explored_nodes, states_visited, goal_node, frontier = solution
    print("--- Greedy solver ---")
    print(" solution_found: True",)
    print(f" nodes_explored: {explored_nodes}\n",
        f"states_visited: {len(states_visited)}\n",
        f"nodes_in_frontier: {frontier.qsize()}\n",
        f"max_depth: {goal_node.depth}\n",
        f"cost_total: {goal_node.state.accumulated_cost}\n",
    )

# -- Breadth first search solver
solution, metrics = breadth_first_search(number_disks=DISKS_NUMBER)
print("--- BFS solver ---")
for key, value in metrics.items():
    print(f"{key}: {value}")

--- Greedy solver ---
 solution_found: True
 nodes_explored: 382
 states_visited: 216
 nodes_in_frontier: 45
 max_depth: 31
 cost_total: 31.0

--- BFS solver ---
solution_found: True
nodes_explored: 1351
states_visited: 233
nodes_in_frontier: 285
max_depth: 31
cost_total: 31.0


7. **Si la solución óptima es de 2k - 1 movimientos (siendo k el número de discos), ¿qué tan lejos está la solución encontrada por el algoritmo implementado de esa solución óptima? (Se recomienda ejecutar al menos 10 veces y usar el promedio de los trayectos obtenidos)**

Tanto en el algoritmo de busqueda implementado como en el provisto por el docente se obtuvo 2^k - 1 movimientos.

In [17]:
%%timeit
solution = greedy_hanoi_solver(number_disks=DISKS_NUMBER)

20.3 ms ± 267 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%timeit
solution = breadth_first_search(number_disks=DISKS_NUMBER)

62.7 ms ± 2.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
tracemalloc.start()
solution = greedy_hanoi_solver(number_disks=DISKS_NUMBER)
_, memory_peak = tracemalloc.get_traced_memory()
memory_peak /= 1024*1024
tracemalloc.stop()
print(f"Pico de memoria ocupada greedy: {round(memory_peak, 2)} [MB]", )

tracemalloc.start()
solution = breadth_first_search(number_disks=DISKS_NUMBER)
_, memory_peak = tracemalloc.get_traced_memory()
memory_peak /= 1024*1024
tracemalloc.stop()
print(f"Pico de memoria ocupada breadth_first: {round(memory_peak, 2)} [MB]", )

Pico de memoria ocupada greedy: 0.28 [MB]
Pico de memoria ocupada breadth_first: 1.44 [MB]


### Conclusiones

Se logro implementar el algoritmo de busqueda informada greedy y resolver el problema de las torres de hanoi. 
Es mas eficiente en todos los aspectos (tiempo de ejecucion, memoria utilizada, cantidad de estados recorridos) que la implementacion de busqueda en anchura.
Se entiende que esto es dependiente de la heuristica utilizada.